In [1]:
import torch

### Prepare input x and initialize h, and c
    x = [Time steps, batch size, No. of features]
    h0, c0= [batch, hidden state features], 0: t-1

In [2]:
ts = 4 #time_steps
bs = 2 # batch_size
f  = 3 # No. of input_features
hf = 5 # No. of hidden_state_features

In [3]:
x = torch.randn(ts, bs, f) # (time step, batch, input_size)
h0 = torch.randn(bs, hf)  # (batch, hidden_size)
c0 = torch.randn(bs, hf) # (batch, hidden_size)
print(x.shape, h0.shape, c0.shape)

torch.Size([4, 2, 3]) torch.Size([2, 5]) torch.Size([2, 5])


### Send x, h0, c0 to a LSTMCell and manually calculate results

In [4]:
lstm = torch.nn.LSTMCell(f, hf) # LSTMCell(input_size, hidden_size)

In [5]:
for name, param in lstm.named_parameters():
    if param.requires_grad:
        print (name, param.data.shape)  #[4*hidden_size, features]

weight_ih torch.Size([20, 3])
weight_hh torch.Size([20, 5])
bias_ih torch.Size([20])
bias_hh torch.Size([20])


In [6]:
Wi = lstm.weight_ih  #(4*hidden_size, input_size)
bi = lstm.bias_ih    #(4*hidden_size)
Wh = lstm.weight_hh  #(4*hidden_size, hidden_size)
bh = lstm.bias_hh    #(4*hidden_size)
print(Wi.shape, bi.shape, Wh.shape, bh.shape)

torch.Size([20, 3]) torch.Size([20]) torch.Size([20, 5]) torch.Size([20])


In [7]:
Wii=Wi[0:5, :]   #row 1, 5, 9, ..., 1+(hidden_size-1)*4
print("Wi =", Wi)
print("Wii = ", Wii)

Wi = Parameter containing:
tensor([[-0.2745,  0.4400, -0.0736],
        [-0.3925, -0.2507, -0.2603],
        [-0.0934, -0.4146, -0.4326],
        [ 0.3296, -0.1157,  0.3513],
        [-0.1616, -0.3468,  0.1701],
        [ 0.2664, -0.1276, -0.2257],
        [ 0.2496,  0.1825, -0.1656],
        [ 0.0864,  0.4458,  0.2742],
        [-0.0738, -0.3542,  0.0487],
        [ 0.2419,  0.2591, -0.1359],
        [-0.3917,  0.2219, -0.0305],
        [ 0.4172, -0.4117,  0.4065],
        [-0.0749,  0.1374, -0.3787],
        [ 0.2996,  0.1550,  0.3194],
        [-0.3594, -0.4053,  0.0205],
        [ 0.3538, -0.0367, -0.1031],
        [ 0.3749, -0.4251, -0.0114],
        [-0.1979, -0.0537, -0.2694],
        [ 0.1589, -0.2054, -0.4468],
        [ 0.4054,  0.2035, -0.3869]], requires_grad=True)
Wii =  tensor([[-0.2745,  0.4400, -0.0736],
        [-0.3925, -0.2507, -0.2603],
        [-0.0934, -0.4146, -0.4326],
        [ 0.3296, -0.1157,  0.3513],
        [-0.1616, -0.3468,  0.1701]], grad_fn=<SliceBackw

In [8]:
Wif=Wi[5:10, :] 
Wig=Wi[10:15, :]   
Wio=Wi[15:20, :]

In [9]:
bii=bi[0:5]
bif=bi[5:10]
big=bi[10:15]
bio=bi[15:20]

Whi=Wh[0:5, :]
Whf=Wh[5:10, :] 
Whg=Wh[10:15, :]   
Who=Wh[15:20, :]   

bhi=bh[0:5]
bhf=bh[5:10]
bhg=bh[10:15]
bho=bh[15:20]

### f(t) = Sigmod( x(t)*Wif + bif + h(t-1)*Whf + bhf )

In [10]:
print(x[0].shape, torch.transpose(Wif, 1, 0).shape) #x= (time step, batch, input_size), so x[0]= x time 0

torch.Size([2, 3]) torch.Size([3, 5])


In [11]:
tmp1 = x[0].mm(torch.transpose(Wif, 1, 0))  # x*Wif
print(tmp1.shape)

torch.Size([2, 5])


In [12]:
# verification
print(x[0])
print(torch.transpose(Wif, 1, 0))
print(tmp1)

tensor([[ 0.2799,  0.2298, -0.2043],
        [ 0.7609, -0.3234,  0.2615]])
tensor([[ 0.2664,  0.2496,  0.0864, -0.0738,  0.2419],
        [-0.1276,  0.1825,  0.4458, -0.3542,  0.2591],
        [-0.2257, -0.1656,  0.2742,  0.0487, -0.1359]],
       grad_fn=<TransposeBackward0>)
tensor([[ 0.0914,  0.1457,  0.0706, -0.1120,  0.1550],
        [ 0.1849,  0.0876, -0.0067,  0.0712,  0.0647]], grad_fn=<MmBackward0>)


In [13]:
tmp1 = x[0].mm(torch.transpose(Wif, 1, 0)) + bif  # x1*Wif + bif
print(tmp1.shape)

torch.Size([2, 5])


In [14]:
# verification
print(bif)
print(tmp1)

tensor([-0.4280, -0.0322,  0.0799, -0.4301, -0.0775], grad_fn=<SliceBackward0>)
tensor([[-0.3366,  0.1135,  0.1505, -0.5421,  0.0775],
        [-0.2430,  0.0555,  0.0732, -0.3590, -0.0128]], grad_fn=<AddBackward0>)


In [15]:
print(h0.shape, Whf.shape)

torch.Size([2, 5]) torch.Size([5, 5])


In [16]:
tmp1 = x[0].mm(torch.transpose(Wif, 1, 0)) + bif  # x1*Wif + bif
tmp2 = h0.mm(Whf) + bhf                            # h0*Whf + bhf
ft = torch.sigmoid(tmp1 + tmp2)
print(ft.shape)

torch.Size([2, 5])


### Similar formula to calculate i(t), g(t), and o(t)

In [17]:
it = torch.sigmoid(x[0].mm(torch.transpose(Wii, 1, 0)) + bii + h0.mm(Whi) + bhi)
gt = torch.tanh(x[0].mm(torch.transpose(Wig, 1, 0)) + big + h0.mm(Whg) + bhg)
ot = torch.sigmoid(x[0].mm(torch.transpose(Wio, 1, 0)) + bio + h0.mm(Who) + bho)
print(it.shape, gt.shape, ot.shape)

torch.Size([2, 5]) torch.Size([2, 5]) torch.Size([2, 5])


### c(t) = f(t) * c(t-1) + i(t) * g(t), where * is element wise multiplication
### h(t) = o(t) * tanh(c(t))

In [18]:
print(ft)
print(c0)
print(ft*c0)

tensor([[0.4840, 0.3736, 0.7973, 0.7003, 0.2937],
        [0.3410, 0.3391, 0.6164, 0.4643, 0.3648]], grad_fn=<SigmoidBackward0>)
tensor([[-0.1870, -1.2337,  0.1734,  1.2796, -0.4159],
        [ 1.3566,  0.4784, -0.3568,  0.8162,  0.3545]])
tensor([[-0.0905, -0.4609,  0.1383,  0.8961, -0.1221],
        [ 0.4627,  0.1622, -0.2199,  0.3790,  0.1293]], grad_fn=<MulBackward0>)


In [19]:
c1=ft*c0 + it*gt
print(c1)

tensor([[-0.2456, -0.7379,  0.3514,  1.1914, -0.0920],
        [ 0.2266,  0.4258, -0.4611,  0.3840, -0.1103]], grad_fn=<AddBackward0>)


In [20]:
h1 = ot* torch.tanh(c1)
print(h1)

tensor([[-0.1761, -0.4754,  0.2014,  0.1065, -0.0617],
        [ 0.0975,  0.2281, -0.1628,  0.1815, -0.0651]], grad_fn=<MulBackward0>)


### Compare with the results directly calculated using PyTorch

In [21]:
h, c = lstm(x[0], (h0, c0)) # time 0's input
print(c)
print(h)

tensor([[-0.5593, -0.7867, -0.7200,  0.5888, -0.2741],
        [ 0.5804,  0.6134, -0.5927,  0.3886,  0.3732]], grad_fn=<AddBackward0>)
tensor([[-0.1720, -0.2963, -0.2549,  0.3150, -0.1921],
        [ 0.2362,  0.2789, -0.3515,  0.1758,  0.1658]], grad_fn=<MulBackward0>)


### Recursive pass time_steps

In [22]:
# from time 1 to time step
output = []
for i in range(x.size(0)):
    h, c = lstm(x[i], (h, c))
    output.append(h)

In [23]:
output

[tensor([[-0.1191, -0.2082, -0.1956,  0.0527, -0.2274],
         [ 0.0793,  0.3053, -0.2101,  0.0822,  0.0775]], grad_fn=<MulBackward0>),
 tensor([[ 0.0480, -0.1115, -0.2015, -0.0053, -0.1540],
         [ 0.0207,  0.2526, -0.2246,  0.0178,  0.0247]], grad_fn=<MulBackward0>),
 tensor([[ 0.0549, -0.0785, -0.2713, -0.1881,  0.1372],
         [ 0.1010, -0.1009, -0.0771, -0.1715,  0.1587]], grad_fn=<MulBackward0>),
 tensor([[-0.0055,  0.2523, -0.2716, -0.1943,  0.2038],
         [ 0.0560, -0.0208, -0.1274, -0.2811,  0.1693]], grad_fn=<MulBackward0>)]